
# 9.时间序列分析                              


#### 笨办法学 Python 数据分析  / learn data analysis the hard way
- @Author：知行并重
- @公众号：知行并重
- @Github 代码：https://github.com/kevin-meng/learn-data-analysis-the-hard-way


## 目录
1. **时间的表示**
    - 三种格式间转换
    
2. **时间信息抽取**
    - 年、月、周、日、时、分、秒
    
3. **日期偏移**
    - 过去x天
    - 月初、月末、季末
    - 日期之间间隔
    
4. **时间窗口滚动聚合**
    - 过去x天内聚合

### Step1: 导入必要的库

In [136]:
import pandas as pd #数据分析
import numpy as np #科学计算

from datetime import datetime,timedelta # 时间处理

### 一、数据读取

股票的交易日期，当天的开盘价、最高价、最低价以及收盘价

In [137]:
df = pd.read_csv("../input/share.csv")

print(df.shape)

df.head()

(237, 6)


,ts_code,trade_date,open,high,low,close
0,000001.SZ,20200430,14.02,14.32,13.88,13.93
1,600000.SH,20200430,10.58,10.79,10.56,10.63
2,601328.SH,20200430,5.17,5.21,5.17,5.18
3,000001.SZ,20200429,13.48,14.10,13.45,14.02
4,600000.SH,20200429,10.35,10.61,10.33,10.61


## 二、基础内容

time包、datetime包以及pandas的datetime模块 都可以用来处理时间。
三者之间关系：

time包（底层）-> datetime包（高层应用封装）-> pandas的datetime模块

## 2.1 时间的表示 主要有三种形式：
1. timestamp时间戳，时间戳表示从1970-1-1 00:00:00开始按**秒**计算的偏移量，表示一个时间点。
2. struct_time时间元组，共有9个元素组。
3. 格式化字符串 time，已格式化的结构使时间更具可读性，包括自定义格式和固定格式，属于string数据类型。[类型格式清单](https://www.runoob.com/python/python-date-time.html)

![](../pics/datetimes.png)

## 2.2 时间格式转化

In [42]:
dt_str = "2019-11-01 11:55:30"
dt_str

'2019-11-01 11:55:30'

### 2.2.1 将格式化字符串 time 转化为struct_time时间元组

可以通过 pd.to_datetime() 或 datetime.strptime()方法。 推荐使用前者，因为前者返回的时间元祖 支持的属性方法更多

In [33]:
# 基于pandas datetime方法
dt = pd.to_datetime(dt_str,format = '%Y-%m-%d %H:%M:%S')
dt

Timestamp('2019-11-01 11:55:30')

In [12]:
# 基于datetime.strptime()方法
# dt = datetime.strptime(dt_str,'%Y-%m-%d %H:%M:%S')

### 2.2.2 将struct_time时间元组 转换为 时间戳time_stamp 

In [9]:
ts = dt.timestamp()
ts

1572609330.0

### 2.2.3 将 时间戳time_stamp 转换为 struct_time时间元组 

In [10]:
dt = datetime.fromtimestamp(ts)
dt

datetime.datetime(2019, 11, 1, 11, 55, 30)

### 2.2.4 将datetime格式转化为形如："20191130" 的日期字符串 
http://strftime.org/

In [8]:
dt_str_new = datetime.strftime(dt,'%Y%m%d')
dt_str_new

'20191101'

## 2.3 时间信息抽取

 抽取 datetime 中的 **日期、时间、年、月、日、小时、分钟、 星期、季度**


### 2.3.1 从datetime格式中提取日期部 和 时间部

In [47]:
dt.date()
# dt.time()

datetime.date(2019, 11, 1)

### 2.3.2 从datetime格式中提取年、月、日、小时、分钟、秒、星期

In [49]:
# dt.year
# dt.month
# dt.day
# dt.hour
# dt.minute
# dt.second
dt.weekday() + 1 # 周几
dt.week          # 第几周

44

## 2.4 日期偏移

偏移参数
-  'years', 'months', 'weeks', 'days','hours', 'minutes', 'seconds',microseconds

In [51]:
from pandas.tseries.offsets import *

### 2.4.1 返回 5天后日期

In [52]:
dt + DateOffset(days = 5)

Timestamp('2019-11-06 11:55:30')

### 2.4.2 返回 10分钟后的日期

In [53]:
dt + DateOffset(minutes = 10)

Timestamp('2019-11-01 12:05:30')

### 2.4.3 月初 月末

常用

In [55]:
# 月初
dt + DateOffset(days = 1-dt.day)
# 月末
dt + DateOffset(months = 1, days = - dt.day)

Timestamp('2019-11-30 11:55:30')

### 2.4.4 季初 季末

In [57]:
# 季初
dt + DateOffset(months = -((dt.month - 1) % 3),days = 1-dt.day)
# 季末
dt + DateOffset(months =3 - ((dt.month - 1) % 3), days = -dt.day)

Timestamp('2019-12-31 11:55:30')

### 2.4.5 年初 年末

In [58]:
# 年初
dt + DateOffset(years = 1- dt.month ,days = 1- dt.day)
# 年末
dt + DateOffset(years =1 ,months =1 - dt.month, days = - dt.day)

Timestamp('2019-12-31 11:55:30')

### 2.4.6 计算两个日期间的间隔天数、月数

计算 "2019-11-01" 与  "2019-5-10" 之间间隔天数

In [63]:
# 前期处理
dt1 = pd.to_datetime( "2019-11-01" ,format = '%Y-%m-%d')
dt2 = pd.to_datetime( "2019-5-10"  ,format = '%Y-%m-%d')

# 间隔天数
day_gap = (dt1-dt2)/np.timedelta64(1,'D')
# 间隔月数
month_gap = (dt1-dt2)/np.timedelta64(1,'M')

## 2.5 时间窗口滚动聚合

shift() 、 rolling()

In [120]:
df['datetime'] = df['trade_date'].apply(lambda x: pd.to_datetime(str(x),format = '%Y-%m-%d'))

# 先排序
df = df.sort_values(['ts_code','datetime'])

In [122]:
df.head()

,ts_code,trade_date,open,high,low,close,datetime
234,000001.SZ,20200102,16.65,16.95,16.55,16.87,2020-01-02
231,000001.SZ,20200103,16.94,17.31,16.92,17.18,2020-01-03
228,000001.SZ,20200106,17.01,17.34,16.91,17.07,2020-01-06
225,000001.SZ,20200107,17.13,17.28,16.95,17.15,2020-01-07
222,000001.SZ,20200108,17.00,17.05,16.63,16.66,2020-01-08


### 2.5.1 前一交易日的最高价

In [127]:
df['lag_high_1'] = df.groupby('ts_code')['high'].shift(1)

In [128]:
df.head()

,ts_code,trade_date,open,high,low,close,datetime,lag_high_1
234,000001.SZ,20200102,16.65,16.95,16.55,16.87,2020-01-02,NaN
231,000001.SZ,20200103,16.94,17.31,16.92,17.18,2020-01-03,16.95
228,000001.SZ,20200106,17.01,17.34,16.91,17.07,2020-01-06,17.31
225,000001.SZ,20200107,17.13,17.28,16.95,17.15,2020-01-07,17.34
222,000001.SZ,20200108,17.00,17.05,16.63,16.66,2020-01-08,17.28


### 2.5.2 每个股票 过去5个交易日的开盘价的均值


In [85]:
pd.Series.rolling??

In [100]:
def mean(ls):
    return np.mean(ls)

In [103]:
out = df.groupby('ts_code').rolling(window = 5,on='datetime').aggregate(mean)[['open']].reset_index()
out

,ts_code,level_1,open
0,000001.SZ,234,NaN
1,000001.SZ,231,NaN
2,000001.SZ,228,NaN
3,000001.SZ,225,NaN
4,000001.SZ,222,16.946
...,...,...,...
232,601328.SH,14,5.156
233,601328.SH,11,5.146
234,601328.SH,8,5.146
235,601328.SH,5,5.142


注：aggregate 可以用使用自定义函数，最大值、最小值等统计方式同理

# 谢谢观看
![](../pics/thankyou.png)
